In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
from backtestad.instruments.irs.ir_swap import InterestRateSwap

InterestRateSwap

backtestad.instruments.irs.ir_swap.InterestRateSwap

In [5]:
from dependency_injector.providers import DependenciesContainer

data_providers = DependenciesContainer()

_interest_rates_data = data_providers.ir_data_provider

In [54]:
from hacksad.misc import today

import pandas as pd

from backtestad.db.sql.interest_rates import get_rate

end_date = today()


rates = get_rate(
    pd.Timestamp("2018-01-01"),
    end_date,
["MOSPRIME", "LIBOR", "IRS", "IRS_ICAAP", "EURIBOR", "OIS_ICAAP"] + ["DEPO_ICAAP"],
    # ["USD", "RUB", "EUR"],
    ["RUB"],
)

# rates["group_importance"] = rates["group_code"].map(SOURCE_PRIORITY)

In [55]:
end_date.date()


from pandas.tseries.offsets import BDay


def get_last_trading_day(date):
    return date + BDay(1) - BDay(1)


date: pd.Timestamp = pd.Timestamp(get_last_trading_day(end_date).date())

rates_today = rates.set_index("date").sort_index().loc[date]
# rates_today = rates.set_index("date").sort_index().loc["2023-11-25"]

In [59]:
group = rates_today.groupby(["date", "base_ccy", "tenor"])
volatility = group.rate.std().dropna()
volatility

date        base_ccy  tenor
2023-11-24  RUB       01M     0.003536
                      01W     0.005020
                      01Y     0.035610
                      02W     0.003889
                      02Y     0.008917
                      03M     0.022274
                      03Y     0.019290
                      04Y     0.010267
                      05Y     0.016801
                      1D      0.052998
Name: rate, dtype: float64

In [44]:
rates[rates.tenor == "01M"]

,base_ccy,tenor,rate,group_code
date,,,,
2023-11-24,RUB,01M,0.074000,OIS_ICAAP


In [61]:
get_rate(
    pd.Timestamp("2023-11-24"),
    end_date,
    ["MOSPRIME", "LIBOR", "IRS", "IRS_ICAAP", "EURIBOR", "OIS_ICAAP"] + ["DEPO_ICAAP"],
    # ["USD", "RUB", "EUR"],
    ["RUB"],
)


2023-11-26 16:07:57,975| ERROR   | Could not resolve IP address for s-msk-v-fxp-gp1, aborting!


BaseSSHTunnelForwarderError: Could not establish session to SSH gateway

In [51]:
{"MOSPRIME", "LIBOR", "IRS", "IRS_ICAAP", "EURIBOR", "OIS_ICAAP"} &{"BASEFUNDING",
 "DEPO_ICAAP",
 "FUNDING RATE",
 "FUNDING_TREASURY_LOANS_RUB",
 "LIQUIDITY_COSTS_CBR",
 "LIQUIDITY_COSTS_FIX3M",
 "LIQUIDITY_COSTS_FLOATON",
 "OIS_ICAAP",
 }


{'OIS_ICAAP'}

In [91]:
end_date.year

2023

01M 0.0035355339059327407
01W 0.005020458146424484
01Y 0.03560989750055454
02W 0.003889087296526015
02Y 0.008916616510762362
03M 0.022273863607376248
03Y 0.01928987299076903
04Y 0.01026719046282866
05Y 0.016800857120992375
1D 0.052997653249931734


In [124]:
from irm.swaption import get_swaption_volatility_data
from hacksad.misc import today
import pandas as pd

df : pd.DataFrame = get_swaption_volatility_data(today() - pd.Timedelta(days=120), today())

In [144]:
from collections import defaultdict
import numpy as np

volatility_res = defaultdict(dict)

for group, d in df.groupby(['tenor', 'group_code']):
    if (std:=d.rate.pct_change().std() * np.sqrt(252) )!=0:
        # volatility_res[group[0]][group[1]] = std
        volatility_res[group[0]] = std

In [145]:
assert  pd.DataFrame(volatility_res).shape[0]==1, "only one ground expected"


ValueError: If using all scalar values, you must pass an index

In [146]:
volatility_res

defaultdict(dict,
            {'01M': 5.9485365479539505,
             '01W': 4.878324208851185,
             '01Y': 3.3278370903047803,
             '02M': 5.537561945271509,
             '02Y': 1.6119216950112998,
             '03M': 5.866909015320375,
             '03Y': 1.942956644491365,
             '04Y': 1.96759178773918,
             '05Y': 1.630485180885455,
             '06M': 5.093555789622191,
             '09M': 3.817544603135831,
             '1D': 0.4040674913531897})

In [127]:
import QuantLib as ql
import numpy as np
from scipy.optimize import minimize

# Define a function to calculate the Black price of a swaption
def black_price(swaption, volatility):
    engine = ql.BlackSwaptionEngine(swaption.termStructure(), ql.QuoteHandle(ql.SimpleQuote(volatility)), ql.NullCalendar())
    swaption.setPricingEngine(engine)
    return swaption.NPV()

# Define the objective function for calibration
def objective_function(params, swaptions, market_vols):
    mean_reversion, volatility = params

    # Create the Hull-White model
    hw_process = ql.HullWhiteProcess(swaptions[0].termStructure(), mean_reversion, volatility)
    hw_model = ql.HullWhite(hw_process)

    # Calculate model-implied volatilities
    model_vols = [np.sqrt(black_price(swaption, volatility) / swaption.annuity(0)) for swaption in swaptions]

    # Calculate the difference between model and market volatilities
    diff = np.array(model_vols) - np.array(market_vols)

    # Return the sum of squared differences
    return np.sum(diff ** 2)

# Sample market data
today = ql.Date(1, 1, 2023)
ql.Settings.instance().evaluationDate = today
rate = ql.QuoteHandle(ql.SimpleQuote(0.02))
swaption_vol = ql.QuoteHandle(ql.SimpleQuote(0.2))
swaption_helpers = []

# Create a dummy yield curve for demonstration purposes
dates = [today + ql.Period(i, ql.Years) for i in range(1, 11)]
rates = [0.02 + 0.001 * i for i in range(1, 11)]
yield_curve = ql.ZeroCurve(dates, rates, ql.Actual360(), ql.NullCalendar(), ql.Linear(), ql.Compounded, ql.Annual)

for i in range(1, 6):
    for j in range(1, 6):
        expiry = ql.Period(i, ql.Years)
        tenor = ql.Period(j, ql.Years)
        helper = ql.SwaptionHelper(expiry, tenor, ql.QuoteHandle(swaption_vol), ql.EuriborSwapIsdaFixA(ql.Period(10, ql.Years)), ql.Period(1, ql.Years), ql.Actual360(), ql.ModifiedFollowing, ql.Euribor3M(), ql.QuoteHandle(rate), ql.Period(2, ql.Days), yield_curve)
        swaption_helpers.append(helper)

# Initial guess for calibration parameters
initial_params = [0.01, 0.1]  # mean reversion, volatility

# Perform calibration
result = minimize(objective_function, initial_params, args=(swaption_helpers, [swaption_vol.value() for _ in swaption_helpers]))

# Extract calibrated parameters
calibrated_mean_reversion, calibrated_volatility = result.x

print(f"Calibrated Mean Reversion: {calibrated_mean_reversion}")
print(f"Calibrated Volatility: {calibrated_volatility}")

# You can now use the calibrated parameters to price swaptions or simulate interest rate paths with the Hull-White model
